![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
## Multi Object Tracking sample
This notebook is an example of how to perform object detection with multi-object tracking (MOT) from a video file to count vehicle traffic. The annotated video is saved into new file.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `inference_host_address` option.

When running this notebook locally, you need to specify your cloud API access token in the [env.ini](../../env.ini) file, located in the same directory as this notebook.

When running this notebook in Google Colab, the cloud API access token should be stored in a user secret named `DEGIRUM_CLOUD_TOKEN`.

In [ ]:
# make sure degirum-tools package and other dependencies are installed
!pip show degirum-tools || pip install degirum-tools

#### Specify where you want to run your inferences, model zoo url, model name and image source

In [ ]:
# inference_host_address: where you want to run inference
#     @cloud to use DeGirum cloud
#     @local to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# model_name: name of the model for running AI inference
# video_source: video source for inference
#     camera index for local camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
# ann_path: path to save annotated video
inference_host_address = "@cloud"
model_zoo_url = "degirum/public"
model_name = "yolo_v5s_coco--512x512_quant_n2x_orca1_1"
video_source = "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/Traffic.mp4"
ann_path = "temp/multi_object_tracking_video_file.mp4"
counting_lines = [(120, 430, 870, 430), (860, 80, 860, 210)]

#### The rest of the cells below should run without any modifications

In [ ]:
import degirum as dg, degirum_tools

# load model
model = dg.load_model(
    model_name=model_name, 
    inference_host_address=inference_host_address,
    zoo_url=model_zoo_url,
    token=degirum_tools.get_token(),
    overlay_color=[(255,0,0)]
)

# create object tracker
tracker = degirum_tools.ObjectTracker(
    class_list=["car"],
    track_thresh=0.35,
    track_buffer=100,
    match_thresh=0.9999,
    trail_depth=20,
    anchor_point=degirum_tools.AnchorPoint.BOTTOM_CENTER,
)

# create line counter
line_counter = degirum_tools.LineCounter(counting_lines)

# attach object tracker and line counter to model
degirum_tools.attach_analyzers(model, [tracker, line_counter])

# annotate video applying object tracker and line counter
degirum_tools.annotate_video(model, video_source, ann_path)

In [ ]:
# display annotated video
degirum_tools.ipython_display(ann_path)

In [ ]:
# display original video
degirum_tools.ipython_display(video_source)